# Loading an existing MODFLOW 6 model and adding a GWT transport model with FloPy

![watershed](images/watershed_overview.png)


A MODFLOW 6 model groundwater transport (GWT) model that simulated groundwater age will be added to an existing groundwater flow (GWF). The model simulation is based on the structured watershed example in [Hughes, J.D., Langevin, C.D., Paulinski, S.R., Larsen, J.D. and Brakenhoff, D. (2024), FloPy Workflows for Creating Structured and Unstructured MODFLOW Models. Groundwater, 62: 124-139. https://doi.org/10.1111/gwat.13327](https://doi.org/10.1111/gwat.13327) and is shown above. The model domain is discretized into 5 layers, 41 rows, and 73 columns and has been modified to add hydraulic conditivity heterogenity in layers 2 and 4. 

Groundwater age will be simulated using a zero-order reaction rate of -1.0 / 365.25, which will simulate concentrations that equal the groundwater age in years. Advection will be simulated using the `upstream` scheme. Dispersion will not be simulated. 

In [ ]:
%matplotlib inline
import pathlib as pl

import flopy
import matplotlib as mpl
import matplotlib.transforms as mtransforms
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import numpy as np

from base import plot_model_results, animate_model_results, ANI_EXT

Before loading the existing MODFLOW 6 groundwater flow model you should define the simulation workspace (`ws`) where the model files are located and the simulation name (`name`). The `ws` should be set to `"simbase"` and `name` should be set to `"watershed"`.

In [ ]:
ws = pl.Path("simbase")
name = "watershed"

## Load the model

Load the existing groundwater flow model using `flopy.mf6.MFSimulation.load()`. Call the simulation object `sim`.

Change the existing simulation workspace to `"class-model"` so we do not alter the existing model files. The simulation workspace can be changed using `sim.set_sim_path(ws)`.

In [ ]:
ws = pl.Path("class-model")


Get the groundwater flow model from the simulation using `sim.get_model()`. The name of the gwf_model is `"gwf"`.

In [ ]:
gwf_name = "gwf"


## Add the groundwater transport model

Create the groundwater transport model object (`gwt`) using `flopy.mf6.ModflowGwf()`. Make sure to include the simulation object (`sim`) as the first variable in the groundwater transport model object and set `modelname` to `gwt`. Use `Shift-Tab` to see the optional variables that can be specified.

If you were to use the same `modelname` for the groundwater transport, you would have to define a unique file name for each package that is also used in the groundwater flow model (`DIS`, `IC`, and `OC`). User specified filenames can be defined when creating an instance of a package using `filename="my_unique_filename.ext`, where `.ext` is the extension for the package (for example, `.dis` for `DIS`, `.ic` for `IC`, _etc._).

In [ ]:
gwt_name = "gwt"
gwt = flopy.mf6.ModflowGwt(
    sim,
    modelname=gwt_name,
    print_input=False,
    save_flows=True,
)


### Create an IMS package for the GWT model

Create the IMS package for the groundwater transport model using `flopy.mf6.ModflowIms()` and register it for use with the groundwater transport model using `sim.register_ims_package()`.

Make sure to define a unique file name for the IMS package using `filename="gwt.ims"` when defining the IMS package for the groundwater transport model.

In [ ]:
imsgwt = flopy.mf6.ModflowIms(
        sim,
        complexity="complex",
        print_option="SUMMARY",
        linear_acceleration="bicgstab",
        outer_maximum=1000,
        inner_maximum=100,
        outer_dvclose=1e-4,
        inner_dvclose=1e-5,
        filename=f"{gwt_name}.ims",
    )

sim.register_ims_package(imsgwt, [gwt_name])

### Create the discretization package for the GWT model

Create the discretization package using `flopy.mf6.ModflowGwtdis()`. Use `Shift-Tab` to see the optional variables that can be specified. A description of the data required by the `DIS` package (`flopy.mf6.ModflowGwtdis()`) can be found in the MODFLOW 6 [ReadTheDocs document](https://modflow6.readthedocs.io/en/latest/_mf6io/gwt-dis.html).

We will extract the discretization data from the groundwater flow model.

In [ ]:
pak = gwf.dis
nlay, nrow, ncol = pak.nlay.array, pak.nrow.array, pak.ncol.array
shape3d = (nlay, nrow, ncol)
xorigin, yorigin = pak.xorigin.array, pak.yorigin.array
delr, delc = pak.delr.array, pak.delc.array
top, botm = pak.top.array, pak.botm.array
idomain = pak.idomain.array

### Create the initial conditions for the GWT model

Create the initial conditions package for the groundwater tansport model using `flopy.mf6.ModflowGwtic()` and set the initial concentration (`strt`) to 0.0.

In [ ]:
# initial conditions
ic = flopy.mf6.ModflowGwtic(
    gwt, 
    strt=0.0, 
    )



### Create the advection package for the GWT model

Create the advection package using `flopy.mf6.ModflowGwtadv()` and set the scheme to `"upstream"`.

In [ ]:
# advection


### Create the mobile storage and transfer package for the GWT model

Create the mobile storage and transfer package using `flopy.mf6.ModflowGwtmst()`. Set the `porosity` to 0.35, `zero_order_decay` to `True`, and `decay` to `-1.0 / 365.25`. A negative decay rate is specified to increase groundwater age with simulation time (increase mass).

### Create the source sink mixing package for the GWT model

Create the source sink mixing package using `flopy.mf6.ModflowGwtssm()`. Add the recharge package (`rch_original`) as a source.

In [ ]:
sourcerecarray = [
    ("rch_original", "AUX", "CONCENTRATION"),
]


### Build output control

Define the output control package (`OC`) for the model using the `flopy.mf6.ModflowGwtoc()` method to `[('CONCENTRATION', 'ALL'), ('BUDGET', 'ALL')]` to save the head and flow for the model. Also the concentration (`concentration_filerecord`) and cell-by-cell flow (`budget_filerecord`) files should be set to `f"{gwt_name}.ucn"` and `f"{gwt_name}.cbc"`, respectively. Use `Shift-Tab` to see the optional variables that can be specified. A description of the data required by the `OC` package (`flopy.mf6.ModflowGwtoc()`) can be found in the MODFLOW 6 [ReadTheDocs document](https://modflow6.readthedocs.io/en/latest/_mf6io/gwt-oc.html).

In [ ]:
# output control
oc = flopy.mf6.ModflowGwtoc(
        gwt,
        budget_filerecord=f"{gwt_name}.cbc",
        concentration_filerecord=f"{gwt_name}.ucn",
        saverecord=[("CONCENTRATION", "ALL"), ("BUDGET", "ALL")],
    )

In [ ]:
gwfgwt = flopy.mf6.ModflowGwfgwt(
    sim,
    exgtype="GWF6-GWT6",
    exgmnamea=gwf_name,
    exgmnameb=gwt_name,
    filename="gwfgwt.exg",
)

### Write the model files and run the model

Write the MODFLOW 6 GWF and GWT model files using `sim.write_simulation()`. Use `Shift-Tab` to see the optional variables that can be specified for `.write_simulation()`.

Run the model using `sim.run_simulation()`, which will run the MODFLOW 6 executable installed in the Miniforge class environment (`pyclass`) and the MODFLOW 6 model files created with `.write_simulation()`. Use `Shift-Tab` to see the optional variables that can be specified for `.run_simulation()`.

## Post-process the results

Load the concentrations and specific discharge from the ucn and cbc files. The concentration file can be loaded with the `gwt.output.concentration().get_data()` method. The cell-by-cell file can be loaded with the `gwf.output.budget().get_data()` method. 

First, we will get the simulation times available in the ucn file using `gwt.output.concentration().get_times()`.


In [ ]:
times = gwt.output.concentration().get_times()

In [ ]:
cobj = gwt.output.concentration()

Retrieve the `'DATA-SPDIS'` data type from the cell-by-cell file. Name the specific discharge data `spd`.

Cell-by-cell data is returned as a list so access the data by using `spd = gwf.output.budget().get_data(text="DATA-SPDIS")[0]`.

In [ ]:
budobj = gwf.output.budget()

Plot the model results using the `plot_model_results()` helper function.

In [ ]:
totim = times[-1]
path = pl.Path(f".figures/result_{totim:06.0f}_l1.png")
plot_model_results(gwf, cobj, budobj, totim=totim, path=path)

Extra Credit: Animate model results

In [ ]:
layer = 0
path = pl.Path(f".animations/result_l{layer + 1}{ANI_EXT}")
vmin, vmax = 0.0, times[-1] / 365.25
animate_model_results(gwf, cobj, budobj, vmin, vmax, path, layer=layer)